In [19]:
import xml
import xml.etree.ElementTree as ET
from os import listdir
from os.path import isfile, join
import os
import re
from collections import defaultdict

In [20]:
os.getcwd()

'/home1/zishan/raghav/notebooks'

In [21]:
mypath='/home1/zishan/raghav/Data/31_Aug_Emotion/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [22]:
print(len(onlyfiles))

124


In [23]:
onlyfiles[:10]

['navbharat_2009_3_6_4231845.xml',
 'in-drought-hit-latur-10000-litres-of-water-for-bjp-minister-helipad2.txt.xml',
 'blast-at-fruit-market-in-northeastern-nigerias-yola-kills-3216.txt.xml',
 'iraq-suicide-bomber-kills-at-least-60-at-checkpoint8.txt.xml',
 'navbharat_2004_8_13_813123.xml',
 'navbharat_2007_6_8_2108083.xml',
 'firozpur-tornado-in-ferozpur-heavy-loss-of-property-175094524.txt.xml',
 'navbharat_2003_8_30_57158709final',
 'navbharat_2009_1_3_3927424.xml',
 'navbharat_2004_9_27_865156']

In [24]:
print(len(onlyfiles))
print(len(set(onlyfiles)))

124
124


In [25]:
import xml.etree.ElementTree as etree

for filename in onlyfiles:
    tree = etree.parse('/home1/zishan/raghav/Data/31_Aug_Emotion/'+str(filename)).getroot()

    results = []

    w_tags = []
    for element in tree.findall('P/*'):
        if element.tag == 'W':
            w_tags.append(element.text)

        for search_element in element.findall('W'):
            w_tags.append(search_element.text)

        if re.match('EMOTION\d-SENTENCE' ,element.tag):
            dict_to_get_class = defaultdict(list)
            for child in element.iter():
                if child.tag=='W':
                    w_tags.append(child.text)
                elif re.match('EMOTION\d-SENTENCE', child.tag):
                    id_sentence = child.tag[7]
                    dict_to_get_class[id_sentence].append(child.attrib['TYPE'])
                elif re.match('EMOTION\d-INTENSITY-SENTENCE', child.tag):
                    id_sentence = child.tag[7]
                    dict_to_get_class[id_sentence].append(float(child.attrib['TYPE']))
            try:
                temp = sorted(dict_to_get_class.items(), key= lambda e: e[1][1], reverse=True)
#                 print(temp)
                label = temp[0][1][0]
#                 print(label)
                intensity = temp[0][1][1]
#                 print(intensity)
                w_tags.append([label, intensity])
                results.append(w_tags)
                w_tags = []
            except Exception as e:
                print('exception in file %s' %(str(filename)))
                print(''.join(w_tags))
                print('*****************')
                w_tags = []
    

    for sentence in results:
        label = sentence[-1]
        words = sentence[:-1]
        words = [word.strip(' ') for word in words]
        with open('/home1/zishan/raghav/Data/data_with_intensity.txt','a+') as f:
            string = ' '.join(words) + '\t' + label[0]+ '\t'+ str(label[1])+'\n'
#             print(string)
            f.write(string.encode().decode())


 

In [11]:
print(results[0])

[' navbharat_2008_5_11_3028752 ', ' तूफान ', ' में ', ' उड़ी ', ' कार, ', ' छत ', ' पर ', ' जा ', ' गिरी ', ['FEAR/ANXIETY', 0.8]]
